In [1]:
import tensorflow as tf
# 导入tensorflow提供的读取MNIST的模块
import tensorflow.examples.tutorials.mnist.input_data as input_data
# 读取MNIST数据
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

C:\Users\Farhan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Farhan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Farhan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Farhan\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# 定义全连接函数

def fcn_layer(inputs, inputs_dim,outputs_dim,activation=None):
#     以截取正太分布的随机数初始化W
    W=tf.Variable(tf.truncated_normal([inputs_dim,outputs_dim],stddev=0.1))
#     以0初始化b
    b=tf.Variable(tf.zeros([outputs_dim]))
    XWb=tf.matmul(inputs,W)+b
    if activation is None: #默认不使用激活函数
        outputs=XWb
    else: #若使用激活函数，则用其对输出结果进行变换
        outputs=activation(XWb)
    return outputs
        

In [3]:
# 构建输入层
x=tf.placeholder(tf.float32,[None,784],name="X")
# 构建隐藏层
H1_NN=256
H2_NN=64
H3_NN=32
h1=fcn_layer(inputs=x,inputs_dim=784,outputs_dim=H1_NN,activation=tf.nn.relu)
h2=fcn_layer(inputs=h1,inputs_dim=H1_NN,outputs_dim=H2_NN,activation=tf.nn.relu)
h3=fcn_layer(inputs=h2,inputs_dim=H2_NN,outputs_dim=H3_NN,activation=tf.nn.relu)
forward=fcn_layer(inputs=h3,inputs_dim=H3_NN,outputs_dim=10,activation=None)
pred=tf.nn.softmax(forward)

In [4]:
# 定义标签数据占位符
y=tf.placeholder(tf.float32,[None,10],name="Y")

In [5]:
# 定义损失函数
loss_function=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=forward, labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
train_epochs=10
batch_size=50
total_batch=int(mnist.train.num_examples/batch_size)
display_step=1
learning_rate=0.01

In [7]:
# 选择优化器
optimizer=tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

In [8]:
correct_prediction=tf.equal(tf.argmax(y,1), tf.argmax(pred,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [11]:
# 记录训练开始时间
from time import time
startTime=time()

sess=tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(train_epochs):
    for batch in range(total_batch):
        xs,ys=mnist.train.next_batch(batch_size)  #读取批次数据
        sess.run(optimizer,feed_dict={x:xs,y:ys})  #执行批次训练
        
    #total_batch个批次训练完成后，使用验证数据计算误差与准确率
    loss,acc=sess.run([loss_function,accuracy],feed_dict={x:mnist.validation.images,y:mnist.validation.labels})
    
    if(epoch+1)% display_step ==0:
        print("Train Epoch:",'%0.2d' %(epoch+1),"Loss=", "{:.9f}".format(loss)," Accuracy=","{:.4f}".format(acc))

# 显示运行总时间
duration=time()-startTime
print("Train Finished takes: ","{:.2f}".format(duration))

Train Epoch: 01 Loss= 0.154361203  Accuracy= 0.9558
Train Epoch: 02 Loss= 0.132121965  Accuracy= 0.9652
Train Epoch: 03 Loss= 0.148151487  Accuracy= 0.9586
Train Epoch: 04 Loss= 0.150666848  Accuracy= 0.9650
Train Epoch: 05 Loss= 0.148753256  Accuracy= 0.9644
Train Epoch: 06 Loss= 0.109876700  Accuracy= 0.9736
Train Epoch: 07 Loss= 0.118844986  Accuracy= 0.9742
Train Epoch: 08 Loss= 0.173799649  Accuracy= 0.9692
Train Epoch: 09 Loss= 0.140083298  Accuracy= 0.9754
Train Epoch: 10 Loss= 0.161133394  Accuracy= 0.9748
Train Finished takes:  34.49
